In [ ]:
def compute_predictions_increment(predictions, class_name, total_score, total_count):
    if class_name in ["nudity", "general_nsfw"]:
        total_score+=predictions[class_name]
        total_count+=1
    return total_score, total_count

def compute_predictions(predictions, total_score, total_count):
    predictions["sex_intent"] = total_score / total_count
    predictions["normal"] = 1 - predictions["sex_intent"]
    return predictions

In [60]:
import requests
import os
import uuid
import json
import shutil
DATASET_API_ACCESS_TOKEN = "ZWbw9GYgHG616YljqLtfDK9FPwZ"
try:
    dataset_api_url = "https://4tro8cx1.directus.app/items/feedbacks?access_token={0}&filter[seen][_eq]=0"
    r = requests.get(dataset_api_url.format(DATASET_API_ACCESS_TOKEN))
    r_json = r.json()["data"]
    path_name = "uploads/{}".format(str(uuid.uuid1()))
    os.mkdir(path_name)
    for feedback in r_json:
        print(feedback)
        r = requests.get(feedback["image_url"], allow_redirects=True)
        file_name = "{0}/{1}.jpg".format(path_name, str(uuid.uuid1()) )
        open(file_name, 'wb').write(r.content)
        updated = {"seen": 1}
        headers = {"Content-Type": "application/json", "Authorization": "Bearer {}".format(DATASET_API_ACCESS_TOKEN)}
        req_url = "https://4tro8cx1.directus.app/items/feedbacks/{}"
        r = requests.patch(req_url.format(feedback["id"]), data=json.dumps(updated), headers=headers)
        break
    send_file(path, as_attachment=True)
    shutil.make_archive(path_name+".zip", 'zip', path_name)
    shutil.rmtree(path_name)
except Exception as wrong:
    print(wrong)
    pass
    

{'id': 29, 'status': 'published', 'sort': None, 'user_created': 'a7336fc3-b951-4601-b220-f07a571e6264', 'date_created': '2022-04-21T15:33:42', 'user_updated': None, 'date_updated': None, 'rating': 0, 'comment': '', 'dataset_id': '5710a157-6e60-4818-a749-c577c85d8164', 'prediction_data': 'female_underwear: 96.94% <br />female_nudity: 95.16% <br />female_swimwear: 98.59% <br />male_shirtless: 60.75% <br />general_nsfw: 64.66% <br />', 'image_url': 'https://images.unsplash.com/photo-1529171109936-dd3412229df6?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=MnwxfDB8MXxyYW5kb218MHw4OTA5NTYwfHx8fHx8fDE2NTA1NTUxMjI&ixlib=rb-1.2.1&q=80&utm_campaign=api-credit&utm_medium=referral&utm_source=unsplash_source&w=1080', 'seen': False, 'class_name': None}
